Topics: Prototyping
* Gradio
* Deploying a Gradio app to Hugging Face
* Using a Gradio API from a Node.js application
* Streamlit

# Chapter 1: Gradio

Simple gradio example

In [3]:
!pip install --quiet gradio

In [12]:

import gradio as gr

def greet(name):
    return "Hello " + name + "!!"

iface = gr.Interface(fn=greet, inputs="text", outputs="text")

if __name__ == '__main__':

    iface.launch()

Running Gradio in a Colab notebook requires sharing enabled. Automatically setting `share=True` (you can turn this off by setting `share=False` in `launch()` explicitly).

Colab notebook detected. To show errors in colab notebook, set debug=True in launch()
* Running on public URL: https://8ebefef551bf54d672.gradio.live

This share link expires in 72 hours. For free permanent hosting and GPU upgrades, run `gradio deploy` from the terminal in the working directory to deploy to Hugging Face Spaces (https://huggingface.co/spaces)


From spacy to gradio

In [13]:
import spacy
import gradio as gr

nlp = spacy.load("en_core_web_sm") # You need to execute python -m spacy download en_core_web_sm, to be able to use this module

# Code adapted from https://journal.code4lib.org/articles/15405
def ner(sentence):
    doc = nlp(sentence)
    places = [ent for ent in doc.ents if ent.label_ in ['GPE', 'LOC']]

    return places

demo = gr.Interface(fn=ner, inputs="text", outputs="text")
demo.launch()


Running Gradio in a Colab notebook requires sharing enabled. Automatically setting `share=True` (you can turn this off by setting `share=False` in `launch()` explicitly).

Colab notebook detected. To show errors in colab notebook, set debug=True in launch()
* Running on public URL: https://832b5bb40011b48289.gradio.live

This share link expires in 72 hours. For free permanent hosting and GPU upgrades, run `gradio deploy` from the terminal in the working directory to deploy to Hugging Face Spaces (https://huggingface.co/spaces)


From LLM to gradio

In [6]:
!pip install -qU langchain langchain-openai

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.0/1.0 MB 37.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 50.4/50.4 kB 4.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 408.7/408.7 kB 22.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.2/1.2 MB 47.3 MB/s eta 0:00:00


In [7]:
import os
import getpass

if "OPENAI_API_KEY" not in os.environ:
    os.environ["OPENAI_API_KEY"] = getpass.getpass("Enter your OpenAI API key: ")

Enter your OpenAI API key: ··········


In [8]:
# From wikipedia: https://en.wikipedia.org/wiki/M%C3%BCnster
example_text = """
Münster is an independent city (Kreisfreie Stadt)
in North Rhine-Westphalia, Germany. It is in the northern part of the state and is considered to
 be the cultural centre of the Westphalia region. It is also a state district capital. Münster was the
  location of the Anabaptist rebellion during the Protestant Reformation and the site of the signing of the
   Treaty of Westphalia ending the Thirty Years' War in 1648. Today, it is known as the bicycle capital of Germany.
Münster gained the status of a Großstadt (major city) with more than 100,000 inhabitants in 1915.[4]
 As of 2014, there are 300,000[5] people living in the city, with about 61,500 students,[6]
 only some of whom are recorded in the official population statistics as having their primary residence in Münster.
 Münster is a part of the international Euregio region with more than 1,000,000 inhabitants (Enschede, Hengelo, Gronau, Osnabrück). Companies offering jobs in
 Münster include the Institute for Geoinformatics at the University of Münster, the Münster University of Applied Sciences, Reedu GmbH, con terra,
 the Deutsche Bank, IKEA, LIDL, REWE, ALDI and BASF Coatings.
"""

In [14]:
from typing_extensions import Annotated, TypedDict
from langchain_openai import ChatOpenAI

llm = ChatOpenAI(temperature=0, model="gpt-4o-mini")

# https://python.langchain.com/docs/how_to/structured_output/#typeddict-or-json-schema
class NamedEntity(TypedDict):
    """Entities to retrieve from the passage."""

    loc: Annotated[str, ..., "All locations in the passage"]
    event: Annotated[str, ..., "All events in the passage"]
    org: Annotated[str, ..., "All organizations in the passage"]

structured_llm = llm.with_structured_output(NamedEntity)

structured_llm.invoke(example_text)

{'loc': 'Münster, North Rhine-Westphalia, Germany',
 'event': "Anabaptist rebellion, signing of the Treaty of Westphalia, Thirty Years' War",
 'org': 'Institute for Geoinformatics, University of Münster, Münster University of Applied Sciences, Reedu GmbH, con terra, Deutsche Bank, IKEA, LIDL, REWE, ALDI, BASF Coatings'}

In [10]:
def ner_extractor(input_text):
  from typing_extensions import Annotated, TypedDict
  from langchain_openai import ChatOpenAI

  llm = ChatOpenAI(temperature=0, model="gpt-4o-mini")

  # https://python.langchain.com/docs/how_to/structured_output/#typeddict-or-json-schema
  class NamedEntity(TypedDict):
      """Entities to retrieve from the passage."""

      loc: Annotated[str, ..., "All locations in the passage"]
      event: Annotated[str, ..., "All events in the passage"]
      org: Annotated[str, ..., "All organizations in the passage"]

  structured_llm = llm.with_structured_output(NamedEntity)

  return structured_llm.invoke(example_text)

In [15]:
import gradio as gr

iface = gr.Interface(fn=ner_extractor, inputs="text", outputs="text")

if __name__ == '__main__':

    iface.launch()

Running Gradio in a Colab notebook requires sharing enabled. Automatically setting `share=True` (you can turn this off by setting `share=False` in `launch()` explicitly).

Colab notebook detected. To show errors in colab notebook, set debug=True in launch()
* Running on public URL: https://81369a94c954f9b5ba.gradio.live

This share link expires in 72 hours. For free permanent hosting and GPU upgrades, run `gradio deploy` from the terminal in the working directory to deploy to Hugging Face Spaces (https://huggingface.co/spaces)


# Chapter 2: Deploying a Gradio app to Hugging Face

* [Example](https://huggingface.co/spaces/aurioldegbelo/ner_space_2023)
* [GitHub repository](https://github.com/aurioldegbelo/gradio-demo)


# Chapter 3: Using a Gradio API from a node.js application

*   [GitHub repository](https://github.com/aurioldegbelo/gradio-node)



# Chapter 4: Streamlit
* [GitHub repository](https://github.com/aurioldegbelo/graphQA-playground)

# Project work

It's now your turn. Pick a strategy 🧰 that fits your skills/interests and start coding 🙂!